In [ ]:
from bs4 import BeautifulSoup as Soup
from IPython.display import HTML
import pandas as pd
import numpy as np
import re
import pprint
from IPython.display import display

In [ ]:
fn = 'orig.procPublicationRequestDMSSPortal.oct-dec-2014.csv'
rows = pd.read_csv(fn, header=0, sep='|', encoding='latin-1')

In [ ]:
cols = ['RequestID', 'StartDate', 'EndDate', 
        'AgencyCode', 'AgencyName', 'AgencyDivision', 
        'TypeOfNoticeCode', 'TypeOfNoticeDescription', 
        'CategoryCode', 'CategoryDescription', 'ShortTitle', 
        'SelectionMethodCode', 'SelectionMethodDescription', 
        'SectionID', 'SectionName', 'SpecialCaseReasonCode', 
        'SpecialCaseReasonDescription', 'PIN', 'DueDate', 
        'AddressToRequest', 'ContactName', 'ContactPhone', 
        'Email', 'AddressToSubmit', 'ContractAmount', 
        'RequestedBy', 'PhoneRequested', 'ConfirmationNumber', 
        'ContactFax', 'AdditionalDescription', 'OtherInfo', 
        'VendorName', 'VendorAddress', 'Printout']

In [ ]:
cas = rows['AgencyName'] == "Citywide Administrative Services"
awards = rows['TypeOfNoticeDescription'] == "Award"
rows = rows[cas & awards][cols]
rows

In [ ]:
from IPython.display import display
badcount = 0
for html in rows.AdditionalDescription.values:
    if not isinstance(html, str):
        badcount += 1
    else:
        display(HTML(html))
        try:
            parse_notice(Soup(html))
        except Exception:
            badcount += 1

In [ ]:
import json
def scrape(row):
    output = None
    try:
        output = parse_notice(Soup(row.AdditionalDescription))
    except Exception :
        output = {'error' : 'bad input: [{}]'.format(row.AdditionalDescription)}
    row['output'] = json.dumps(output)
    return row

In [ ]:
fix = rows.apply(scrape,1)

In [ ]:
for rec in fix[['output', 'AdditionalDescription']].values:
    output, text = rec
    if not isinstance(text, str):
        text = 'NAN'
    display(HTML(text))
    print('---')
    pprint.pprint(json.loads(output))
    display(HTML('<hr />'))
    print('\n\n')